# 3. Import biosphere3 and ecoinvent

Now that you know how to work with the foreground system, it's time to learn how to work with the background system. In particular it is useful to import and play around with two databases: _biosphere3_ that contains all the exchanges and impact assessment methods, and _ecoinvent_. You need to get the ecoinvent files first, see steps below:

1. Open the ecoinvent website and [login](https://www.ecoinvent.org/login-databases.html) with your username and password.
2. You should read somewhere: _To download LCI and LCIA cumulative matrices click here_. Click there.
3. Select _ecoinvent 3.6._
4. Download the file `ecoinvent 3.6_consequential_ecoSpold02.7z` in a folder of yours. Make sure you remember the full path to this directory. E.g. I have downloaded the file in:
_/Users/massimo/Documents/Databases/ecoinvent v3.6_
5. The file you have downloaded is a compressed archive of many files (like with winzip or winrar). Extract the files from the .7z archive, e.g. by double clicking it. If it does not work, install a software that can do that. E.g. for mac users you can either download [theunarchiver](https://theunarchiver.com/) or, if you are using brew, just open terminal and do `brew install p7zip` and then from terminal find the folder and do `7z x 'ecoinvent 3.6_consequential_ecoSpold02.7z'` (here the [p7zip instructions](https://wiki.archlinux.org/index.php/p7zip) in case).
6. Now you an run the cells below. Make sure you change the path line and replace it with the one where you have extracted the files. For example, I have extracted the files in a folder called "datasets". The path to this folder is: _/Users/massimo/Documents/Databases/ecoinvent v3.6/datasets_ You will see this same line in the script and you need to change it with your directory. 

In [1]:
import brightway2 as bw

In [2]:
bw.projects # check what project you have 
#bw.projects.delete_project('advlca21', delete_dir=True) # if you want a fresh start

Brightway2 projects manager with 41 objects, including:
	B4B18
	BTC02
	BTC_Att_LCA
	Biowood
	C3BO
	CCU
	ConseqUncertainty
	GSA
	Giovanni
	HH
Use `sorted(projects)` to get full list, `projects.report()` to get
	a report on all projects.

In [3]:
bw.projects.delete_project('advlca23', delete_dir = True)

'default'

In [4]:
bw.projects.set_current('advlca23') # Still working in the same project
bw.databases
#bw.databases.clear() # For a fresh start (Risky command! clears all your existing databases)

Databases dictionary with 0 objects

Before importing ecoinvent, we need to make a default setup of Brightway2. This means importing all the environmental exchanges and all the LCIA methods. Then when we import ecoinvent the ecoinvent activities will be linked to this database of environmentla exchanges, just like in the previous example wiht the product system of H&S 2002.

In [5]:
# Import the biosphere3 database
bw.bw2setup() # This will take some time

Creating default biosphere



Writing activities to SQLite3 database:


Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.01 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 02/10/2023 10:48:48
  Finished: 02/10/2023 10:48:48
  Total time elapsed: 00:00:00
  CPU %: 97.20
  Memory %: 1.19
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Applied 5 strategies in 1.82 seconds
Wrote 975 LCIA methods with 254388 characterization factors
Creating core data migrations



We are going to use version 3.6 of ecoinvent, consequential model, for this course.

In [15]:
# Import ecoinvent

# You need to change the line below with the directory where you have saved ecoinvent
ei36dir = "/Users/massimo/Documents/Databases/ecoinvent v3.6/datasets"

if 'ecoinvent 3.6 conseq' in bw.databases:
    print("Database has already been imported")
else:
    ei36 = bw.SingleOutputEcospold2Importer(ei36dir, 'ecoinvent 3.6 conseq') # You can give it another name of course
    ei36.apply_strategies()
    ei36.statistics()

ei36.drop_unlinked(True)
ei36.write_database() # This will take some time.

Extracting XML data from 16459 datasets
Extracted 16459 datasets in 74.79 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
3 exchanges couldn't be linked and were deleted. See the logfile for details:
	/Users/massimo/Library/Logs/Brightway3/advlca23.c7ffca2b3f03634666c75e56

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:41


Title: Writing activities to SQLite3 database:
  Started: 02/10/2023 10:58:29
  Finished: 02/10/2023 10:59:10
  Total time elapsed: 00:00:41
  CPU %: 91.70
  Memory %: 30.13
Created database: ecoinvent 3.6 conseq


Brightway2 SQLiteBackend: ecoinvent 3.6 conseq

In [18]:
bw.databases # you should now see both "biosphere3" and "ecoinvent 3.6 conseq"

Databases dictionary with 2 object(s):
	biosphere3
	ecoinvent 3.6 conseq

# Navigate biosphere3 and ecoinvent

A key difference compared to previous exercises is that in ecoinvent each activity and exchange is defined by a **code** which are unique identifiers. So it is important to learn how to find both activity code and name and how to match them _(Actually we used the codes also in the previous lectures but they were identical to the activity names for simplicity)_.

In [19]:
# Search stuff in biosphere
bw.Database("biosphere3").search("carbon dioxide") # there is more than one activity with this name. Only code is univocal.

['Carbon dioxide, fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, in air' (kilogram, None, ('natural resource', 'in air')),
 'Carbon dioxide, fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air',)),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'non-urban air or from high stacks')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'lower stratosphere + upper troposphere')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'low population density, long-term')),
 'Carbon dioxide, non-fossil' (kilogram, None, ('air', 'urban air close to ground')),
 'Carbon dioxide, to soil or biomass stock' (kilogram, None, ('soil', 'agricultura

In [20]:
CO2 = bw.Database("biosphere3").get("349b29d1-3e58-4c66-98b9-9d1a076efd2e") # This code works across bw2 installations, 
                                                                    ### i.e. is univocal for biosphere3 everywhere
print(CO2['name']) # there is more than one activity with this name. Only code is univocal.
print(CO2['code'])

Carbon dioxide, fossil
349b29d1-3e58-4c66-98b9-9d1a076efd2e


In [21]:
# Search stuff in ecoinvent

# Search by keyword
mydb = bw.Database('ecoinvent 3.6 conseq')
#mydb.search("*") # to search everything
mydb.search("transport freight euro5")

#bw.Database('ecoinvent 3.6 conseq').search("transport") # gives the same result obviously

['market for transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RER, None),
 'market for transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RER, None),
 'market for transport, freight, lorry 16-32 metric ton, EURO5' (ton kilometer, RER, None),
 'market for transport, freight, lorry 3.5-7.5 metric ton, EURO5' (ton kilometer, RER, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry >32 metric ton, EURO5' (ton kilometer, RER, None),
 'transport, freight, lorry 3.5-7.5 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry 7.5-16 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry 16-32 metric ton, EURO5' (ton kilometer, RoW, None),
 'transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lorry, unspecified' (ton kilometer, RoW, None),
 'transport, freight, lorry, all sizes, EURO5 to generic market for transport, freight, lor

In [22]:
activity_name = 'electricity'

# Same but different:
for activity in bw.Database("ecoinvent 3.6 conseq").search(activity_name, limit = 5):  
    print(activity)
    print(activity['code'])

'electricity production, oil' (kilowatt hour, CO, None)
0ea8277efafe376b0b19a9306611e6a6
'electricity production, oil' (kilowatt hour, CN-GZ, None)
c17a357fddef38c7471acfa7ef53fedf
'electricity production, oil' (kilowatt hour, CN-FJ, None)
0182b3744343ba95e78472fed594e6e9
'electricity production, oil' (kilowatt hour, CN-GX, None)
3ae3a2cc51ae6dd8bf90f62db52e921b
'electricity production, oil' (kilowatt hour, LU, None)
d7b174d31e5f94e5120affcb9e210698


In [23]:
# Try this        
for activity in bw.Database("biosphere3").search(activity_name):  
    print(activity)
    print(activity['code']) # Can you explain this result?

'Exported energy - electricity' (megajoule, None, ('inventory indicator', 'output flow'))
3a813213-cdbc-4bd7-a888-f75fbda0538a


In [24]:
# you can be much more specific in your search:
for activity in bw.Database("ecoinvent 3.6 conseq").search(activity_name, filter={"location" : 'DK'}, limit = 5):
    print(activity)
    print(activity['code'])

Excluding 3077 filtered results
'electricity production, oil' (kilowatt hour, DK, None)
4fffb800b04aa36192982edea677002d
'market for electricity, low voltage' (kilowatt hour, DK, None)
7f0d57e31e5b9767f04d12719a872b7c
'market for electricity, high voltage' (kilowatt hour, DK, None)
8210bddcdd0f18de07449a356c462200
'market for electricity, medium voltage' (kilowatt hour, DK, None)
35889622d7d53fc5e8a6c4b7ee5ca6a5
'electricity, high voltage, import from SE' (kilowatt hour, DK, None)
6c53c5acaa870a75a2d74373a541d284


Now you know how to find activities. What about **selecting** activities?

In [25]:
# If you know the code (e.g. found with method above) it's simple.        
mycode = '7f0d57e31e5b9767f04d12719a872b7c'
myact = bw.Database("ecoinvent 3.6 conseq").get(mycode)
#myact = Database("biosphere3").get(mycode)  # Not working of course...

print(myact['name'])

market for electricity, low voltage


In [26]:
myact._data # a lot of detail

{'comment': 'This dataset describes the electricity available on the low voltage level in {{location}} for year {{year}}. This is done by showing the transmission of 1kWh electricity at low voltage.\nTechnology:  Average technology used to transmit and distribute electricity. Includes underground and overhead lines, as well as air-, vacuum- and SF6-insulated high-to-medium voltage switching stations. Electricity production according to related technology datasets.\nTime period:  The composition of this market is valid  for the year 2014.',
 'classifications': [('EcoSpold01Categories', 'electricity/supply mix'),
  ('ISIC rev.4 ecoinvent',
   '3510:Electric power generation, transmission and distribution'),
  ('CPC', '17100: Electrical energy')],
 'activity type': 'market activity',
 'activity': 'bf26854a-5049-4f32-897b-04be30861c62',
 'database': 'ecoinvent 3.6 conseq',
 'filename': 'bf26854a-5049-4f32-897b-04be30861c62_d69294d7-8d64-4915-a896-9996a014c410.spold',
 'location': 'DK',
 'n

In [27]:
for i in list(myact.exchanges()):  # Epxlore the activity as usual
    print(i['type'])
    print(i)
    print(i['input'])
    print('-------')

production
Exchange: 1.0 kilowatt hour 'market for electricity, low voltage' (kilowatt hour, DK, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.6 conseq', '7f0d57e31e5b9767f04d12719a872b7c')
-------
technosphere
Exchange: 6.27e-09 kilogram 'market for sulfur hexafluoride, liquid' (kilogram, RER, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.6 conseq', 'b5d6ab027d8556912a0880c107d24213')
-------
technosphere
Exchange: 8.74048809653223e-08 kilometer 'market for distribution network, electricity, low voltage' (kilometer, GLO, None) to 'market for electricity, low voltage' (kilowatt hour, DK, None)>
('ecoinvent 3.6 conseq', '586a94df3a0c67ee0fa46a4e76e3507d')
-------
technosphere
Exchange: 7.36096869893038e-05 kilowatt hour 'electricity production, photovoltaic, 3kWp slanted-roof installation, multi-Si, panel, mounted' (kilowatt hour, DK, None) to 'market for electricity, low voltage' (kilowatt hour, DK, Non

In [28]:
# If you know the name and want to select it:
activity_name = 'market for electricity, low voltage'
    
for activity in bw.Database("ecoinvent 3.6 conseq"):  # can you find an easier way? I couldn't
    if activity['name'] == activity_name:
        myact = bw.Database("ecoinvent 3.6 conseq").get(activity['code'])

myact  # Careful! Might not return the danish market. Not what I wanted! 

'market for electricity, low voltage' (kilowatt hour, PH, None)

In [29]:
for activity in bw.Database("ecoinvent 3.6 conseq"):  
    if activity['name'] == activity_name and activity['location'] == "DK":  # need to be specific...
        myact = bw.Database("ecoinvent 3.6 conseq").get(activity['code'])
myact  # alright

'market for electricity, low voltage' (kilowatt hour, DK, None)

# Calculate with biosphere3 and ecoinvent 

Ok now we can run an LCA with a dataset from ecoinvent.

In [30]:
list(bw.methods)

[('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 500a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 20a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 100a'),
 ('CML 2001 (superseded)', 'malodours air', 'malodours air'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 20a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP infinite'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 500a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 500a'),
 ('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 20a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP'),
 ('CML 2001 (superseded)', 'land use', 'competition'),
 ('CML 2001 (superseded)', 'acidification potential'

In [31]:
# First select a method from the list (use 'list(methods)' to see all of them)
mymethod = ('IPCC 2013', 'climate change', 'GWP 100a')
mymethod

('IPCC 2013', 'climate change', 'GWP 100a')

In [32]:
myact = bw.Database("ecoinvent 3.6 conseq").get('7f0d57e31e5b9767f04d12719a872b7c')
myact

'market for electricity, low voltage' (kilowatt hour, DK, None)

In [33]:
functional_unit = {myact : 1}
lca = bw.LCA(functional_unit, mymethod) #run LCA calculations again with method
lca.lci()
lca.lcia()
print(lca.score) # What is the unit? Find out! (alright it's bw.methods[mymethod])

0.04282987668572083


In [34]:
bw.methods[mymethod]

{'description': "IPCC characterisation factors for the direct global warming potential of air emissions in ecoinvent 3.2. See the ecoinvent report 'Implementation of IPCC impact assessment method 2007 and 2013 to ecoinvent database 3.2 (2015.11.30)' for more details. Doesn't include: indirect formation of dinitrogen monoxide from nitrogen emissions, radiative forcing due to emissions of NOx, water, sulphate, etc. in the lower stratosphere + upper troposphere, carbon climate feedbacks, and any emissions which have cooling effects (e.g. black carbon). All CO is assumed to convert completely to CO2. Biogenic CO2 uptake and biogenic CO2 emissions are not characterised, except for 'Carbon dioxide, from soil or biomass stock' (deforestation and land transformation).",
 'filename': 'LCIA_Implementation_3.8.xlsx',
 'unit': 'kg CO2-Eq',
 'abbreviation': 'ipcc-2013cg.bd5af3f67229a1cc291b8ecb7f316fcf',
 'num_cfs': 211}

# Exercise (at home)
Link the emissions of your previously defined foreground system to the biosphere3 database, and link some of the ecoinvent database activities to your foreground system. Run the calculations and get a carbon footprint with the ILCD climate change method.